# Schedulers

Some hyperparameter optimisation algorithms are written as scheduling algorithms.

They terminate, pause, clone and alter hyperparameters of running trial.

In [ ]:
import time
from ray import train, tune
from ray.tune.schedulers import ASHAScheduler

def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation == "relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

def objective(config):
    """
    Evaluates score of experiment in training loop
    Uses train.report to report score back to Tune
    """
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        train.report({
            "iterations": step,
            "mean_loss": score
        })

### ASHA Scheduler

Parallel hyperparameter tuning, exploit parallelism and aggressive early-stopping.  

In [13]:
from ray.tune.schedulers import ASHAScheduler

asha_scheduler = ASHAScheduler(
    time_attr="training_iteration",
    metric="mean_loss",
    mode="min",
    max_t=100,
    grace_period=10,
    reduction_factor=3,
    brackets=1,
)

tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(scheduler=asha_scheduler, num_samples=10),
    param_space={
        "steps": 100,
        "width": tune.uniform(0, 20),
        "height": tune.uniform(-100, 100),
        "activation": tune.choice(["relu", "tanh"]),
    },
)

In [14]:
results = tuner.fit()

2024-12-03 18:16:32,449	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/sidharrthnagappan/ray_results/objective_2024-12-03_18-16-18' in 0.0100s.
2024-12-03 18:16:32,455	INFO tune.py:1041 -- Total run time: 13.16 seconds (13.14 seconds for the tuning loop).


In [18]:
results.get_best_result(metric="mean_loss", mode="max")

Result(
  metrics={'iterations': 9, 'mean_loss': 13.521147076761155},
  path='/Users/sidharrthnagappan/ray_results/objective_2024-12-03_18-16-18/objective_b1866_00003_3_activation=relu,height=29.5986,width=18.6846_2024-12-03_18-16-19',
  filesystem='local',
  checkpoint=None
)